In [1]:
import numpy as np

```
x1 -----\
         \
          \
          
          sig(w1 * x1 + w2 * x2 + b) ---- O (L)
          /
         /
x2 -----/        
```

here `O` is the output of the perceptron and `L` is the loss.

## Ref: Sigmoid Gradient
$$\sigma(x) = \frac{1}{1+e^{-x}}$$

$$\frac{\delta \sigma(x)}{\delta x} = \sigma(x)(1 - \sigma(x))$$

NOTE: Other gradients can be easily calculated

---
# Generic class definitions for various operators/nodes

In [2]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [3]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value * self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient

In [4]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value + self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = 1 * self.output.gradient

In [5]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(1/(1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

---
# Defining the computational graph for a Perceptron

In [6]:
def forward_nn():
    # w1 * x1
    w1x1 = w1_mul_x1.forward(w1, x1)
    # w2 * x2
    w2x2 = w2_mul_x2.forward(w2, x2)
    # w1*x1 + w2*x2
    w1x1_w2x2 = w1x1_add_w2x2.forward(w1x1, w2x2)
    # w1*x1 + w2*x2 + b
    w1x1_w2x2_b = w1x1w2x2_add_b.forward(w1x1_w2x2, b)
    # sigmoid(w1*x1 + w2*x2 + b)
    output = sigmoid_out.forward(w1x1_w2x2_b)
    return output

def backward_nn():
    sigmoid_out.backward()
    w1x1w2x2_add_b.backward()
    w1x1_add_w2x2.backward()
    w2_mul_x2.backward()
    w1_mul_x1.backward()

### Validation

In [8]:
# Initialize Weights and Bias
w1 = Node(0.1, 0.0)
w2 = Node(0.4, 0.0)
b = Node(-0.02, 0.0)

# Input/Target Output
alpha = 0.001
x1 = Node(0.3, 0.0)
x2 = Node(1.0, 0.0)
y = 0.475

# Create Nodes
w1_mul_x1 = MultiplyNode()
w2_mul_x2 = MultiplyNode()
w1x1_add_w2x2 = AddNode()
w1x1w2x2_add_b = AddNode()
sigmoid_out = SigmoidNode()

In [9]:
for i in range(100000):
    forward_output = forward_nn()
    forward_output.gradient = -2 * (y - forward_output.value)
    backward_nn()
    w1.value -= alpha * w1.gradient
    w2.value -= alpha * w2.gradient
    b.value -= alpha * b.gradient
    
forward_output.value

0.47500000000065534

---
# Putting it all together into a single Perceptron

In [10]:
class Perceptron(object):
    def __init__(self, x1, x2, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha
        ### Initializing weights/bias to a random float between -1 and 1.
        self.w1 = Node(np.random.uniform(-1, 1), 0.0)
        self.w2 = Node(np.random.uniform(-1, 1), 0.0)
        self.b = Node(np.random.uniform(-1, 1), 0.0)
        ### Input and Output variables
        self.x1 = Node(x1, 0.0)
        self.x2 = Node(x2, 0.0)
        ### Initialize operators nodes required 
        ### for processing the inputs within a perceptron
        self.initialize_operators()
    
    def initialize_operators(self):
        self.w1_mul_x1 = MultiplyNode()
        self.w2_mul_x2 = MultiplyNode()
        self.w1x1_add_w2x2 = AddNode()
        self.w1x1_w2x2_add_b = AddNode()
        self.sigmoid = SigmoidNode()
    
    def forward(self):
        w1x1 = self.w1_mul_x1.forward(self.w1, self.x1)
        w2x2 = self.w2_mul_x2.forward(self.w2, self.x2)
        w1x1_w2x2 = self.w1x1_add_w2x2.forward(w1x1, w2x2)
        w1x1_w2x2_b = self.w1x1_w2x2_add_b.forward(w1x1_w2x2, self.b)
        self.sigmoid.forward(w1x1_w2x2_b)
        
    def backward(self):
        self.sigmoid.backward()
        self.w1x1_w2x2_add_b.backward()
        self.w1x1_add_w2x2.backward()
        self.w2_mul_x2.backward()
        self.w1_mul_x1.backward()
    
    def update(self):
        self.w1.value -= self.alpha * self.w1.gradient
        self.w2.value -= self.alpha * self.w2.gradient
        self.b.value -= self.alpha * self.b.gradient

### Validation

In [11]:
p = Perceptron(0.11, -1.0, alpha=0.1)
# number of iterations
N = 100000
# expected output 
target = 0.3481972639817

for i in range(N):
    # Step 1. Forward Pass
    p.forward()
    # Step 2. Calculate Loss. -2 * (y - output) is the gradient of output w.r.t 
    # square loss function.
    p.sigmoid.output.gradient = -2 * (target - p.sigmoid.output.value)
    # Step 3. Backward Pass
    p.backward()
    # Step 4. Update Weights and Bias
    p.update()

p.sigmoid.output.value

0.34819726398170059

---

# Assignment 3

```
X1 ------\
          \
          (P) ---- (P) ---- (T)
          /
X2 ------/
         
```

1. Create a two layer neural network with one perceptron in each layer (see Diagram above). Write a validation code that does along with your implementation. The goal of network is to optimize the two perceptrons to produce the output target `T` given the inputs `X1` and `X2`. Assume the output `O` of each perceptron is

$$ O = \sigma{(w1*x1 + w2*x2 + b)} $$ where

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Feel free to change the loss function if you like.



In [34]:
class Perceptron2(object):
    def __init__(self, x1, x2, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha
        ### Initializing weights/bias to a random float between -1 and 1.
        self.w1 = Node(np.random.uniform(-1, 1), 0.0)
        self.b = Node(np.random.uniform(-1, 1), 0.0)
        ### Input and Output variables
        self.x1 = Perceptron(x1, x2, alpha=alpha)
        ### Initialize operators nodes required 
        ### for processing the inputs within a perceptron
        self.initialize_operators()
    
    def initialize_operators(self):
        self.w1_mul_x1 = MultiplyNode()
        self.w1x1_add_b = AddNode()
        self.sigmoid = SigmoidNode()
    
    def forward(self):
        self.x1.forward()
        #print(self.x1.sigmoid)
        #print(self.x1.sigmoid.output.value)
        w1x1 = self.w1_mul_x1.forward(self.w1, self.x1.sigmoid.output)

        w1x1_b = self.w1x1_add_b.forward(w1x1, self.b)
        self.sigmoid.forward(w1x1_b)
        
    def backward(self):
        self.sigmoid.backward()
        self.w1x1_add_b.backward()
        self.w1_mul_x1.backward()
        self.x1.backward()
    
    def update(self):
        self.w1.value -= self.alpha * self.w1.gradient
        self.b.value -= self.alpha * self.b.gradient
        self.x1.update()

In [35]:
#p = Perceptron(0.11, -1.0, alpha=0.1)
p2 = Perceptron2(0.11, -1.0, alpha=0.1)
# number of iterations
N = 100000
# expected output 
target = 0.3481972639817

for i in range(N):
    # Step 1. Forward Pass
    #p.forward()
    # Step 2. Calculate Loss. -2 * (y - output) is the gradient of output w.r.t 
    # square loss function.
    p2.forward()
    p2.sigmoid.output.gradient = -2 * (target - p2.sigmoid.output.value)
    #p.sigmoid.output.gradient = -2 * (target - p.sigmoid.output.value)
    p2.backward()
    # Step 3. Backward Pass
    #p.backward()
    # Step 4. Update Weights and Bias
    p2.update()
    #p.update()

p2.sigmoid.output.value

0.3481972639817002